# Import packages + functions

In [1]:
import os
import sys
import math
import random
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch import nn
import torchvision
from torchvision import transforms
from torchvision.models.feature_extraction import create_feature_extractor
from tqdm import tqdm
import PIL
from datetime import datetime
import h5py
import webdataset as wds
from subprocess import call

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device:",device)

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

device: cpu


# Load NSD webdatasets for a given subject

The '/fsx' directory refers to a directory housed on the Stability HPC. You will need to download NSD data from [our huggingface dataset](https://huggingface.co/datasets/pscotti/naturalscenesdataset/tree/main) and change the folders below to your own folder path. If you have demonstrated serious involvement with the project and need access to the Stability HPC contact Paul Scotti on Discord.

In [2]:
# Preprocessing for images
transform = transforms.Compose([
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # alexnet requirement
])

def preprocess(sample):
    voxel, image = sample
    return voxel, transform(image)

In [3]:
subject = '01'

num_devices = torch.cuda.device_count()
if num_devices==0: # not using gpu, assuming you want to use the limited dataset
    num_devices = 1
    num_workers = 1
    batch_size = 300
    num_samples = 9000
else: # using cuda, assuming you want to use the full dataset
    num_workers = num_devices
    batch_size = 300
    num_samples = 9000 # see metadata.json
print("num_devices",num_devices)
print("num_workers",num_workers)
print("batch_size",batch_size)
global_batch_size = batch_size * num_devices
print("global_batch_size",global_batch_size)
num_batches = math.floor(num_samples / global_batch_size)
num_worker_batches = math.floor(num_batches / num_workers)
print("num_worker_batches",num_worker_batches)
# train_url = f'/fsx/proj-medarc/fmri/natural-scenes-dataset/webdataset/train/train_subj{subject}'
train_url = f'/scratch/gpfs/KNORMAN/webdataset_nsd/webdataset_split/train/train_subj{subject}'
train_url = train_url+"_{0..49}.tar"

train_data = wds.DataPipeline([wds.ResampledShards(train_url),
                    wds.tarfile_to_samples(),
                    #wds.shuffle(500,initial=500),
                    wds.decode("torch"),
                    wds.rename(images="jpg;png", voxels="nsdgeneral.npy", trial="trial.npy"),
                    wds.to_tuple("voxels", 'images'),
                    wds.map(preprocess),
                    wds.batched(batch_size, partial=True),
                ]).with_epoch(num_worker_batches)
train_dl = torch.utils.data.DataLoader(train_data, batch_size=None, num_workers=num_workers, shuffle=False)

# Validation #
# num_samples = 492
num_samples = 300
num_batches = math.ceil(num_samples / global_batch_size)
num_worker_batches = math.floor(num_batches / num_workers)
print("validation: num_worker_batches",num_worker_batches)

# val_url = f'/fsx/proj-medarc/fmri/natural-scenes-dataset/webdataset/val/val_subj{subject}_0.tar'
val_url = f'/scratch/gpfs/KNORMAN/webdataset_nsd/webdataset_split/val/val_subj{subject}_0.tar'
val_data = wds.DataPipeline([wds.SimpleShardList(val_url),
                    wds.tarfile_to_samples(),
                    wds.decode("torch"),
                    wds.rename(images="jpg;png", voxels="nsdgeneral.npy", trial="trial.npy"),
                    wds.to_tuple("voxels", 'images'),
                    wds.map(preprocess),
                    wds.batched(batch_size, partial=True),
                ]).with_epoch(num_worker_batches)
val_dl = torch.utils.data.DataLoader(val_data, batch_size=None, num_workers=num_workers, shuffle=False)

num_devices 1
num_workers 1
batch_size 300
global_batch_size 300
num_worker_batches 30
validation: num_worker_batches 1


In [13]:
# check that your training data loaders are working
for train_i, (voxel, img_input) in enumerate(train_dl):
    print("idx",train_i)
    print("voxel.shape",voxel.shape,voxel[0,20:24])
    print("img_input.shape",img_input.shape,img_input[0,0,0,100:104])
    if train_i>2:
        break

idx 0
voxel.shape torch.Size([300, 15724]) tensor([ 1.0732, -0.1417, -0.8462,  0.4048], dtype=torch.float16)
img_input.shape torch.Size([300, 3, 256, 256]) tensor([-2.0152, -1.9295, -1.8097, -1.7925])
idx 1
voxel.shape torch.Size([300, 15724]) tensor([-0.0723, -0.9531, -1.3066, -2.9062], dtype=torch.float16)
img_input.shape torch.Size([300, 3, 256, 256]) tensor([ 0.8276, -0.3883, -0.5082, -0.6452])
idx 2
voxel.shape torch.Size([300, 15724]) tensor([ 0.5205, -0.2817, -1.7275,  0.3120], dtype=torch.float16)
img_input.shape torch.Size([300, 3, 256, 256]) tensor([1.8722, 1.8550, 1.8037, 1.8037])
idx 3
voxel.shape torch.Size([300, 15724]) tensor([ 0.0998,  0.3765, -0.5747, -0.2568], dtype=torch.float16)
img_input.shape torch.Size([300, 3, 256, 256]) tensor([-0.6452, -0.7308, -0.7650, -0.8335])


In [14]:
# check that validation data loader is working
for val_i, (voxel, img_input) in enumerate(tqdm(val_dl)):
    print("idx",val_i)
    print("voxel.shape",voxel.shape,voxel[0,20:24])
    print("img_input.shape",img_input.shape,img_input[0,0,0,100:104])
    if train_i>2:
        break

0it [00:02, ?it/s]

idx 0
voxel.shape torch.Size([300, 15724]) tensor([ 1.0420,  1.2803, -0.4688,  1.0654], dtype=torch.float16)
img_input.shape torch.Size([300, 3, 256, 256]) tensor([-0.1999, -0.0972, -0.0801, -0.0458])


In [15]:
# check validation batches are working correctly
# nothing prints out if correct
for train_i, (voxel, img_input) in enumerate(tqdm(val_dl)):
    if voxel.shape[0]!=batch_size:
        print("idx",train_i)
        print("voxel.shape",voxel.shape,voxel[0,20:24])
        print("img_input.shape",img_input.shape,img_input[0,0,0,100:104])
        break

1it [00:02,  2.60s/it]


# Train model (input = image; output = predicted fMRI)

### load pretrained alexnet

In [16]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet') # run in commandline if not loading
model.to(device) # send the model to the chosen device ('cpu' or 'cuda')
model.eval() # set the model to evaluation mode, since you are not training it

Using cache found in /home/dw26/.cache/torch/hub/pytorch_vision_v0.10.0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

### Extract and downsample chosen alexnet features

In [17]:
# load model
model_layer = "features.5"  # choose layer
feature_extractor = create_feature_extractor(model, return_nodes=[model_layer])

In [18]:
print(feature_extractor)

AlexNet(
  (features): Module(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
)



def forward(self, x : torch.Tensor):
    features_0 = getattr(self.features, "0")(x);  x = None
    features_1 = getattr(self.features, "1")(features_0);  features_0 = None
    features_2 = getattr(self.features, "2")(features_1);  features_1 = None
    features_3 = getattr(self.features, "3")(features_2);  features_2 = None
    features_4 = getattr(self.features, "4")(features_3);  features_3 = None
    features_5 = getattr(self.features, "5")(features_4);  features_4 = None
    return {'features.5': features_5}
    


In [19]:
# Incremental PCA computes features
from sklearn.decomposition import IncrementalPCA

def fit_pca(feature_extractor, dataloader):

    # Define PCA parameters
    pca = IncrementalPCA(n_components=100, batch_size=batch_size)
    # Fit PCA to batch
    for _, (voxel, img) in enumerate(tqdm(dataloader)):
        # Extract features
        ft = feature_extractor(img.to(device))
        # Flatten the features
        ft = torch.hstack([torch.flatten(l, start_dim=1) for l in ft.values()])
        # Fit PCA to batch
        pca.partial_fit(ft.detach().cpu().numpy())
    return pca

In [1]:
pca = fit_pca(feature_extractor, train_dl)

NameError: name 'fit_pca' is not defined

In [ ]:
# gets principal components of features for each images using derived PCA
# while going through dataset, also prepare training/validation fMRI matrix from dataloader for regression
def extract_features(feature_extractor, dataloader, pca):

    features = []
    fmri = torch.tensor([])
    for _, (voxel, img) in enumerate(tqdm(dataloader)):
        # Extract features
        ft = feature_extractor(img.to(device))
        # Flatten the features
        ft = torch.hstack([torch.flatten(l, start_dim=1) for l in ft.values()])
        # Apply PCA transform
        ft = pca.transform(ft.cpu().detach().numpy())
        features.append(ft)
        
        fmri = torch.cat((fmri, voxel), 0)
    return np.vstack(features), fmri

In [ ]:
features_train, fmri_train = extract_features(feature_extractor, train_dl, pca)
features_val, fmri_val = extract_features(feature_extractor, val_dl, pca)

print('\nTraining images features:')
print(features_train.shape)
print('(Training stimulus images × PCA features)')

print('\nValidation images features:')
print(features_val.shape)
print('(Validation stimulus images × PCA features)')

In [ ]:
del model, pca

### mask for nsdgeneral voxels

In [ ]:
# whole brain space is 81x104x83
# nsdgeneral sapce is 15724
import nibabel as nib

file = f"/scratch/gpfs/KNORMAN/natural-scenes-dataset/nsddata/ppdata/subj{subject}/func1pt8mm/roi/nsdgeneral.nii.gz"
nifti = nib.load(file)
nsd_mask = nifti.get_fdata()
nsd_mask = nsd_mask>0
nsd_mask = torch.from_numpy(nsd_mask)
print(nsd_mask.shape)
print(torch.sum(nsd_mask))

### Linearly map downsampled alexnet features to fMRI

In [ ]:
print("fMRI training shape:",fmri_train.shape)
print("fMRI validation shape:",fmri_val.shape)
print("Linearized Features shape:", features_train.shape)

In [ ]:
print(fmri_train[0])

In [ ]:
from sklearn.linear_model import Ridge

# Fit linear regressions on the training data
reg = Ridge().fit(features_train, fmri_train)
# Use fitted linear regressions to predict the validation and test fMRI data
fmri_val_pred = reg.predict(features_val)

In [ ]:
print(fmri_val_pred.shape)
print(fmri_val.shape)

# Evaluations

In [ ]:
from scipy.stats import pearsonr as corr

correlation = np.zeros(fmri_val_pred.shape[1])
# Correlate each predicted voxel with the corresponding ground truth voxel
for v in tqdm(range(fmri_val_pred.shape[1])):
    correlation[v] = corr(fmri_val_pred[:,v], fmri_val[:,v])[0]

In [ ]:
print(correlation.shape)
print(correlation)

In [ ]:
# Get roi class for given roi
def getRoiClass(roi):
    if roi in ["V1v", "V1d", "V2v", "V2d", "V3v", "V3d", "hV4"]:
        roi_class = 'prf-visualrois'
    elif roi in ["EBA", "FBA-1", "FBA-2", "mTL-bodies"]:
        roi_class = 'floc-bodies'
    elif roi in ["OFA", "FFA-1", "FFA-2", "mTL-faces", "aTL-faces"]:
        roi_class = 'floc-faces'
    elif roi in ["OPA", "PPA", "RSC"]:
        roi_class = 'floc-places'
    elif roi in ["OWFA", "VWFA-1", "VWFA-2", "mfs-words", "mTL-words"]:
        roi_class = 'floc-words'
    elif roi in ["early", "midventral", "midlateral", "midparietal", "ventral", "lateral", "parietal"]:
        roi_class = 'streams'
    elif roi in ["nsdgeneral"]:
        roi_class = 'nsdgeneral'
    else:
        raise ValueError('invalid ROI')
    return roi_class

In [ ]:
# returns nsdgeneral roi mask
def getRoiMask(hemisphere, roi):
    # get ROI class for given ROI
    roi_class = getRoiClass(roi)
    file_roi = f"/scratch/gpfs/KNORMAN/natural-scenes-dataset/nsddata/ppdata/subj{subject}/func1pt8mm/roi/{hemisphere[0]}h.{roi_class}.nii.gz"
    nifti_roi = nib.load(file_roi)
    mask_roi = nifti_roi.get_fdata()
    
    # get integer indices for ROI's mask in each ROI class
    file_map = f"/scratch/gpfs/KNORMAN/natural-scenes-dataset/nsddata/freesurfer/subj{subject}/label/{roi_class}.mgz.ctab"
    with open(file_map, "r") as file1:
        map_list = file1.readlines()
        map_dict = {}
        for label in map_list:
            roi_label = label.split()
            map_dict.update({roi_label[1]: int(roi_label[0])})

    label_int = map_dict[roi]
    mask_roi[mask_roi!=label_int] = 0
    mask_roi = np.asarray(mask_roi,bool)
    mask_roi = mask_roi[nsd_mask==1]
    
    return mask_roi

In [ ]:
# get correlation by ROI
roi_list = ["V1v", "V1d", "V2v", "V2d", "V3v", "V3d", "hV4", "EBA", "FBA-1", "FBA-2", "mTL-bodies", "OFA", "FFA-1", "FFA-2", "mTL-faces", "aTL-faces", "OPA", "PPA", "RSC", "OWFA", "VWFA-1", "VWFA-2", "mfs-words", "mTL-words", "early", "midventral", "midlateral", "midparietal", "ventral", "lateral", "parietal", "nsdgeneral"]

lh_roi_correlation = []
rh_roi_correlation = []
for hemisphere in ['left', 'right']:
    for roi in tqdm(roi_list):
        mask_roi = getRoiMask(hemisphere,roi)
        correlation_roi = correlation[mask_roi==1]

        if hemisphere == 'left':
            lh_roi_correlation.append(correlation_roi)
        else:
            rh_roi_correlation.append(correlation_roi)
            

In [ ]:
# Create the plot
lh_median_roi_correlation = [np.median(lh_roi_correlation[r]) for r in range(len(lh_roi_correlation))]
rh_median_roi_correlation = [np.median(rh_roi_correlation[r]) for r in range(len(rh_roi_correlation))]
plt.figure(figsize=(18,6))
x = np.arange(len(roi_list))
width = 0.30
plt.bar(x - width/2, lh_median_roi_correlation, width, label='Left Hemisphere')
plt.bar(x + width/2, rh_median_roi_correlation, width,label='Right Hemishpere')
plt.xlim(left=min(x)-.5, right=max(x)+.5)
plt.ylim(bottom=0, top=1)
plt.xlabel('ROIs')
plt.xticks(ticks=x, labels=roi_list, rotation=60)
plt.ylabel('Median Pearson\'s $r$')
plt.legend(frameon=True, loc=1);

# Map from native voxel space to fsaverage surface space
The Algonauts competition requires results in fsaverage space.

## Save voxel predictions as 3d Nifti file

In [ ]:
print(correlation.shape)
print(correlation)

In [ ]:
from nilearn.image import load_img, new_img_like
# temp_s3_dir = '/fsx/proj-medarc/fmri/natural-scenes-dataset/temp_s3'
temp_s3_dir = '/scratch/gpfs/KNORMAN/natural-scenes-dataset'

roi = roi_list[7]
print(roi)

mask_roi = getRoiMask('left',roi)
print(mask_roi.shape)
print(np.sum(mask_roi.astype(int)))
# data_to_convert = np.copy(fmri_val_pred[0]) # example activation
data_to_convert = np.copy(correlation) # correlations

data_to_convert[mask_roi==False]=0
mask = torch.clone(nsd_mask).numpy()
mask[mask==True]=data_to_convert
data_to_convert = mask
data_to_convert = np.array(data_to_convert).astype(np.float32)
# data_to_convert = np.moveaxis(data_to_convert,0,-1)

out_niimg_path = 'out.nii.gz'
avg_out_niimg_path = 'avg_out.nii.gz'

ref_niimg = load_img(f'{temp_s3_dir}/nsddata_betas/ppdata/subj{subject}/func1pt8mm/betas_fithrf_GLMdenoise_RR/R2_session01.nii.gz')
assert ref_niimg.get_fdata().shape == data_to_convert.shape[:3]

out_niimg = new_img_like(ref_niimg, data_to_convert, affine=None, copy_header=True)
out_niimg.to_filename(out_niimg_path)
print(out_niimg.shape)

# also output a nifti volume averaging across the first dimension
# avg_out_niimg = new_img_like(ref_niimg, np.mean(data_to_convert,axis=3), affine=None, copy_header=True)
avg_out_niimg = new_img_like(ref_niimg, np.mean(data_to_convert,axis=0), affine=None, copy_header=True)
avg_out_niimg.to_filename(avg_out_niimg_path)
print(avg_out_niimg.shape)

### Visualize brain volume (sanity check)

In [ ]:
from nilearn import plotting
anat = load_img(f'{temp_s3_dir}/nsddata/ppdata/subj01/anat/T1_0pt8_masked.nii.gz')
# plotting.plot_anat(anat).add_overlay(avg_out_niimg,cmap=plotting.cm.cold_hot,clim=[-3,3])
plotting.plot_anat(anat).add_overlay(out_niimg,cmap=plotting.cm.cold_hot,clim=[-3,3])
# plotting.view_img(avg_out_niimg,bg_img=anat)

## Convert Nifti from native voxel space -> native surface space -> fsaverage surface space

In [ ]:
from nsdcode.nsd_mapdata import NSDmapdata
from nsdcode.nsd_datalocation import nsd_datalocation
nsd = NSDmapdata(temp_s3_dir)

# Example terminal commands to cp NSD buckets temporarily to local storage 
# remove "--recursive" if copying a single file instead of contents of a folder
# in_dir = 'nsddata/ppdata/subj01/transforms/lh.MNI-to-layerB3.mgz'
# print(f"aws s3 cp --recursive s3://natural-scenes-dataset/{in_dir} {temp_s3_dir}/{in_dir}")

In [ ]:
# Mapping to subject-specific surface space across the three cortical depths
for hemisphere in ['lh','rh']:
    data = []
    for p in range(3):
        data.append(
            nsd.fit(
                int(subject),
                'func1pt8', # source space
                f'{hemisphere}.layerB{p+1}', # target space
                out_niimg_path,
                'cubic',
                badval=0
            )
        )
    data = np.array(data)
    assert np.all(data==0) == False

    # Now we average results across the three cortical depths and use
    # nearest-neighbor interpolation to bring the result to fsaverage.
    surface_data = nsd.fit(
        int(subject),
        f'{hemisphere}.white',
        'fsaverage',
        np.mean(data, axis=0),
        interptype=None,
        badval=0,
        fsdir=f'{temp_s3_dir}/nsddata/freesurfer/subj01')
    if hemisphere=='lh':
        lh_surface_data = surface_data
    elif hemisphere=='rh':
        rh_surface_data = surface_data
    print(f"hemisphere {hemisphere}: surface_data.shape=", surface_data.shape)
    
lh_surface_data = lh_surface_data.astype(np.float32)
rh_surface_data = rh_surface_data.astype(np.float32)

assert not np.all(lh_surface_data == rh_surface_data)

In [ ]:
print(data.shape)

## Visualize surface activations

In [ ]:
# Create the interactive brain surface map
from nilearn import datasets
for hemisphere in ['left','right']:
    if hemisphere=='left':
        surface_data = lh_surface_data
    elif hemisphere=='right':
        surface_data = rh_surface_data
    # fsaverage = datasets.fetch_surf_fsaverage('fsaverage')
    fsaverage = datasets.fetch_surf_fsaverage('fsaverage', data_dir='/scratch/gpfs/dw26')
    print(surface_data.shape)
    plotting.plot_surf(
        surf_mesh=fsaverage['infl_'+hemisphere],
#         surf_map=np.mean(surface_data,axis=1),
        surf_map = surface_data,
        bg_map=fsaverage['sulc_'+hemisphere],
        hemi=hemisphere,
        view='lateral',
        threshold=1e-14,
        cmap='cold_hot',
        colorbar=True,
        title='All vertices, '+hemisphere+' hemisphere'
        )

# Save results in a way compatible with Algonauts submission
After predicted data of all subjects and hemispheres have been saved, zip the parent submission folder (algonauts_2023_challenge_submission), which can then be submitted to CodaLab.

In [ ]:
np.save(f'algonauts_2023_challenge_submission/subj{subject}/lh_pred_test.npy',lh_surface_data)
np.save(f'algonauts_2023_challenge_submission/subj{subject}/rh_pred_test.npy',rh_surface_data)
print("Saved predictions!")